In [1]:
import requests
from uuid import uuid4

In [2]:
appID = "YOUR_APP_ID"

In [5]:
url = 'https://api.einvoice.nat.gov.tw/PB2CAPIVAN/invapp/InvApp'

In [14]:
import requests
from requests.exceptions import RequestException
from functools import wraps
import time

def retry_request(retries=10):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(retries):
                try:
                    response = func(*args, **kwargs)
                    response.raise_for_status()
                    return response
                except (RequestException, Exception) as e:
                    print(f"Request failed with error: {e}")
            raise Exception(f"Request failed after {retries} retries")

        return wrapper

    return decorator

@retry_request(retries=50)
def send_request(url, data):
    response = requests.post(url, data=data, timeout=(3, 1))
    return response

try:
    data = {'version': '0.6', 
            'type': 'Barcode',
            'invNum': 'MB46891606',
            'action': 'qryInvDetail',
            'generation': 'V2',
            'invTerm': '11204',
            'invDate': '2023/04/08',
            'UUID': str(uuid4()),
            'randomNumber': '7828',
            'appID': appID
    }
    response = send_request(url, data)
    print(response.text)
except Exception as e:
    print(f"Failed to send request: {e}")


Request failed with error: HTTPSConnectionPool(host='api.einvoice.nat.gov.tw', port=443): Max retries exceeded with url: /PB2CAPIVAN/invapp/InvApp (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fd00801ab20>, 'Connection to api.einvoice.nat.gov.tw timed out. (connect timeout=3)'))
{"msg":"執行成功","amount":"129","code":"200","invNum":"MB46891606","invoiceTime":"13:11:22","invStatus":"已確認","sellerName":"台灣善商股份有限公司古亭門市","invPeriod":"11204","sellerAddress":"台北市中正區羅斯福路二段50號1樓","sellerBan":"38545381","buyerBan":"","currency":"","details":[{"unitPrice":"129","amount":"129","quantity":"1","rowNum":"1","description":"牛丼 中 ^○青Ｓ"}],"invDate":"20230408"}
